In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
from collections import deque
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from xgboost import cv
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer
from math import sqrt

# Divided


## XGBoost


In [3]:
minus = pd.read_csv('advanced_minus_data_for_ml.csv', index_col=0)
plus = pd.read_csv('advanced_data_for_ml.csv', index_col=0)
train = pd.read_csv('train.csv', index_col=0)
minus.loc[train.index, 'class'] = train['class']
plus.loc[train.index, 'class'] = train['class']
plus['D_mean'] = plus.apply(lambda x: x.D_sum / x.D_count, axis=1)
plus['K_mean'] = plus.apply(lambda x: x.K_sum / x.K_count, axis=1)
plus['D_standard_error'] = plus.apply(lambda x: sqrt(x.D_squarred_difference / (x.D_count - 1)), axis = 1)
plus['K_standard_error'] = plus.apply(lambda x: sqrt(x.K_squarred_difference / (x.K_count - 1)), axis = 1)

minus['D_mean'] = minus.apply(lambda x: x.D_sum / x.D_count, axis=1)
minus['K_mean'] = minus.apply(lambda x: x.K_sum / x.K_count, axis=1)
minus['D_standard_error'] = minus.apply(lambda x: sqrt(x.D_squarred_difference / (x.D_count - 1)), axis = 1)
minus['K_standard_error'] = minus.apply(lambda x: sqrt(x.K_squarred_difference / (x.K_count - 1)), axis = 1)

/tmp/ipykernel_48680/4212510273.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['D' 'K' 'D' ... 'K' 'D' 'K']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  minus.loc[train.index, 'class'] = train['class']
/tmp/ipykernel_48680/4212510273.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['D' 'K' 'D' ... 'K' 'D' 'K']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  plus.loc[train.index, 'class'] = train['class']


In [4]:
plus

,class,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,K_squarred_difference,K_standard_error,K_max,K_min
id,,,,,,,,,,,,,,,
0,D,777.614282,2975.333333,0.261354,146.102883,0.221633,0.934560,8.721873e-06,189.940639,1218.666667,0.155859,51.886187,0.206425,0.708587,5.579978e-06
1,K,379.111613,2749.833333,0.137867,309.572835,0.335589,0.919535,8.830171e-08,320.586678,1293.166667,0.247908,118.153121,0.302387,0.712269,3.714698e-07
2,D,464.316126,2942.166667,0.157814,80.203118,0.165134,0.888571,6.482814e-07,156.871831,1235.500000,0.126970,38.416845,0.176407,0.678869,4.606541e-06
3,K,294.214732,2791.833333,0.105384,126.880890,0.213222,0.663689,6.930774e-07,211.299259,1292.666667,0.163460,56.289564,0.208756,0.838197,9.555784e-09
4,D,820.776927,3009.000000,0.272774,156.927662,0.228408,0.922264,2.884368e-06,196.406639,1249.500000,0.157188,54.868411,0.209637,0.840227,1.146566e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,NaN,892.810953,3019.000000,0.295731,162.570080,0.232092,0.855965,2.215619e-07,194.093758,1227.333333,0.158143,59.578361,0.220414,0.793329,2.313791e-08
14996,NaN,307.139350,2754.333333,0.111511,212.117017,0.277561,0.587058,4.618590e-07,273.660614,1323.500000,0.206770,79.489755,0.245165,0.750197,4.231970e-06
14997,NaN,367.227889,2820.833333,0.130184,125.487351,0.210954,0.709202,5.492385e-06,219.240044,1303.333333,0.168215,52.525153,0.200827,0.612782,2.085316e-06


### Plus


In [5]:
X, y = plus.loc[0:9999].drop(columns=['class']), plus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = plus.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1212)
X_train

/tmp/ipykernel_48680/310420619.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = plus.loc[0:9999].drop(columns=['class']), plus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,K_squarred_difference,K_standard_error,K_max,K_min
id,,,,,,,,,,,,,,
6924,733.571073,3007.666667,0.243900,110.082679,0.191345,0.823977,1.347232e-06,168.244496,1223.000000,0.137567,41.178858,0.183570,0.566338,4.233310e-07
3218,258.404984,2780.166667,0.092946,79.275184,0.168893,0.815628,1.340428e-07,160.409217,1284.500000,0.124881,33.101027,0.160592,0.571127,7.548239e-09
6505,350.642826,2746.833333,0.127653,269.013429,0.313004,0.707243,5.213691e-07,295.814766,1291.833333,0.228988,106.580799,0.287345,0.688148,3.946085e-06
6914,576.206706,3007.833333,0.191569,91.986383,0.174907,0.780528,2.877029e-08,132.365749,1234.000000,0.107266,28.451621,0.151905,0.657304,1.171013e-06
5450,747.772729,2972.500000,0.251564,147.286794,0.222635,0.882589,8.267095e-06,179.604189,1250.833333,0.143588,52.410125,0.204777,0.691539,3.996129e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,396.871263,2679.833333,0.148096,397.752356,0.385331,0.917331,3.526281e-08,358.504243,1298.666667,0.276056,160.211406,0.351370,0.912905,2.429452e-07
2626,839.154667,3035.166667,0.276477,138.955299,0.214002,0.849217,2.607821e-06,193.694831,1240.166667,0.156185,55.939920,0.212469,0.842770,5.896064e-07
7606,420.633505,2964.833333,0.141874,63.279746,0.146119,0.757752,5.341530e-07,153.079281,1288.500000,0.118804,29.179225,0.150544,0.601871,9.265371e-08


In [6]:
X_val

,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,K_squarred_difference,K_standard_error,K_max,K_min
id,,,,,,,,,,,,,,
551,792.994341,3051.833333,0.259842,149.573264,0.221420,0.877975,8.188233e-06,202.865289,1250.500000,0.162227,60.195495,0.219490,0.769319,3.878509e-06
2743,295.676851,2802.333333,0.105511,74.223150,0.162775,0.792788,2.147891e-07,156.975110,1284.500000,0.122207,30.336020,0.153738,0.676404,5.715693e-07
7051,379.128083,2934.000000,0.129219,62.735400,0.146251,0.798627,6.649372e-08,131.588459,1230.333333,0.106954,28.781678,0.153011,0.845406,2.068348e-07
7945,652.817843,3028.166667,0.215582,131.677541,0.208563,0.812617,1.937590e-07,209.896134,1265.666667,0.165838,58.087413,0.214315,0.762088,1.313304e-06
7976,496.583288,2637.333333,0.188290,724.185335,0.524113,0.899930,3.465448e-07,478.868471,1279.500000,0.374262,257.550143,0.448829,0.915503,5.393416e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,736.670664,2975.333333,0.247593,145.855375,0.221445,0.863724,2.704056e-07,175.671996,1219.500000,0.144052,51.837082,0.206256,0.847445,2.108207e-06
1982,402.772199,2688.166667,0.149832,425.645833,0.397994,0.779561,3.153591e-07,369.456179,1288.000000,0.286845,161.040350,0.353735,0.837359,1.273905e-06
1871,608.053160,2905.000000,0.209313,114.089235,0.198209,0.884370,3.592658e-07,171.750164,1243.333333,0.138137,41.649424,0.183099,0.706477,2.519930e-07


In [7]:
clf = XGBClassifier(n_estimators = 5000, learning_rate = 0.01, max_depth = 2, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61457	validation_1-logloss:0.62198
[1]	validation_0-logloss:0.61241	validation_1-logloss:0.61979
[2]	validation_0-logloss:0.61031	validation_1-logloss:0.61766
[3]	validation_0-logloss:0.60825	validation_1-logloss:0.61557
[4]	validation_0-logloss:0.60624	validation_1-logloss:0.61353
[5]	validation_0-logloss:0.60428	validation_1-logloss:0.61155
[6]	validation_0-logloss:0.60236	validation_1-logloss:0.60960
[7]	validation_0-logloss:0.60048	validation_1-logloss:0.60769
[8]	validation_0-logloss:0.59863	validation_1-logloss:0.60581
[9]	validation_0-logloss:0.59682	validation_1-logloss:0.60396
[10]	validation_0-logloss:0.59506	validation_1-logloss:0.60219
[11]	validation_0-logloss:0.59333	validation_1-logloss:0.60041
[12]	validation_0-logloss:0.59163	validation_1-logloss:0.59868
[13]	validation_0-logloss:0.58998	validation_1-logloss:0.59702
[14]	validation_0-logloss:0.58835	validation_1-logloss:0.59535
[15]	validation_0-logloss:0.58675	validation_1-logloss:0.59372
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=2,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5000,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [8]:
test_accuracy = accuracy_score(y_val, clf.predict(X_val))
print(f"Test accuracy: {test_accuracy:.12f}")

Test accuracy: 0.781500000000


### Minus


In [9]:
X, y = minus.loc[0:9999].drop(columns=['class']), minus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = minus.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1212)
X_train

/tmp/ipykernel_48680/388348891.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = minus.loc[0:9999].drop(columns=['class']), minus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,K_squarred_difference,K_standard_error,K_max,K_min
id,,,,,,,,,,,,,,
6924,-383.983402,2763.000000,-0.138973,324.215517,0.342614,-6.437497e-07,-0.646349,-346.120014,1346.833333,-0.256988,117.445332,0.295408,-1.423239e-05,-0.870466
3218,-364.500352,2897.000000,-0.125820,52.295786,0.134380,-1.232915e-06,-0.928284,-127.461724,1250.000000,-0.101969,23.742645,0.137874,-1.575889e-06,-0.625178
6505,-633.585025,2896.333333,-0.218754,107.949864,0.193091,-1.403129e-07,-0.747243,-153.978012,1226.166667,-0.125577,39.046006,0.178522,-1.055533e-06,-0.577677
6914,-297.821153,2780.666667,-0.107104,207.068255,0.272936,-1.821257e-07,-0.756640,-270.438693,1343.833333,-0.201244,81.831194,0.246859,-3.571162e-07,-0.698927
5450,-417.377456,2817.500000,-0.148138,293.412884,0.322764,-1.689035e-06,-0.830458,-317.997614,1330.500000,-0.239006,112.941419,0.291462,-1.514036e-06,-0.830112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,-789.833391,2882.833333,-0.273978,172.498723,0.244657,-1.045246e-06,-0.890257,-189.426389,1209.500000,-0.156615,62.058680,0.226609,-1.563298e-07,-0.780176
2626,-424.327595,2778.500000,-0.152718,380.225569,0.369993,-3.958245e-06,-0.814381,-375.366193,1355.000000,-0.277023,135.979167,0.316903,-6.225933e-06,-0.761067
7606,-337.197303,2879.500000,-0.117103,98.535436,0.185018,-1.759998e-08,-0.618574,-196.913304,1329.000000,-0.148167,41.770204,0.177351,-3.668782e-07,-0.752201


In [10]:
X_val

,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,K_squarred_difference,K_standard_error,K_max,K_min
id,,,,,,,,,,,,,,
551,-449.596535,2810.500000,-0.159970,345.548640,0.350703,-2.967524e-06,-0.740864,-362.984396,1367.166667,-0.265501,129.373059,0.307730,-8.560637e-06,-0.825129
2743,-359.419068,2903.000000,-0.123810,58.426562,0.141892,-5.806634e-07,-0.850722,-138.537018,1254.666667,-0.110417,27.935962,0.149276,-1.778487e-06,-0.661282
7051,-272.502854,2757.000000,-0.098840,94.839918,0.185505,-1.741706e-08,-0.819120,-175.953896,1312.333333,-0.134077,39.280614,0.173074,-1.863411e-07,-0.703409
7945,-461.242724,2834.333333,-0.162734,261.374942,0.303727,-1.953604e-06,-0.841380,-317.696880,1360.666667,-0.233486,99.512789,0.270535,-8.349905e-06,-0.810057
7976,-999.827115,2864.166667,-0.349081,217.676149,0.275729,-1.269402e-08,-0.968897,-223.262296,1176.500000,-0.189768,84.701097,0.268431,-5.650230e-07,-0.846103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,-387.150528,2757.666667,-0.140391,359.539639,0.361145,-3.203011e-06,-0.654477,-357.323985,1336.166667,-0.267425,139.851560,0.323643,-6.053008e-06,-0.833649
1982,-791.727115,2896.166667,-0.273371,153.348066,0.230145,-1.814357e-07,-0.890188,-182.451662,1209.333333,-0.150870,54.864592,0.213085,-2.839285e-06,-0.754098
1871,-382.533684,2775.666667,-0.137817,208.007168,0.273800,-1.067679e-06,-0.655150,-272.766367,1304.833333,-0.209043,86.808803,0.258030,-6.712774e-08,-0.720010


In [11]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 5, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61404	validation_1-logloss:0.62164
[1]	validation_0-logloss:0.61138	validation_1-logloss:0.61915
[2]	validation_0-logloss:0.60877	validation_1-logloss:0.61672
[3]	validation_0-logloss:0.60622	validation_1-logloss:0.61435
[4]	validation_0-logloss:0.60373	validation_1-logloss:0.61199
[5]	validation_0-logloss:0.60129	validation_1-logloss:0.60977
[6]	validation_0-logloss:0.59890	validation_1-logloss:0.60753
[7]	validation_0-logloss:0.59652	validation_1-logloss:0.60533
[8]	validation_0-logloss:0.59419	validation_1-logloss:0.60318
[9]	validation_0-logloss:0.59191	validation_1-logloss:0.60108
[10]	validation_0-logloss:0.58966	validation_1-logloss:0.59911
[11]	validation_0-logloss:0.58745	validation_1-logloss:0.59716
[12]	validation_0-logloss:0.58529	validation_1-logloss:0.59531
[13]	validation_0-logloss:0.58318	validation_1-logloss:0.59338
[14]	validation_0-logloss:0.58110	validation_1-logloss:0.59161
[15]	validation_0-logloss:0.57907	validation_1-logloss:0.58984
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [12]:
test_accuracy = accuracy_score(y_val, clf.predict(X_val))
print(f"Test accuracy: {test_accuracy:.12f}")

Test accuracy: 0.776500000000


### Both


In [13]:
cols = minus.drop('class', axis=1).columns.tolist()
renme = {}
for i in range(len(cols)):
    renme[cols[i]] = 'minus_' + cols[i]
renme

{'D_sum': 'minus_D_sum',
 'D_count': 'minus_D_count',
 'D_mean': 'minus_D_mean',
 'D_squarred_difference': 'minus_D_squarred_difference',
 'D_standard_error': 'minus_D_standard_error',
 'D_max': 'minus_D_max',
 'D_min': 'minus_D_min',
 'K_sum': 'minus_K_sum',
 'K_count': 'minus_K_count',
 'K_mean': 'minus_K_mean',
 'K_squarred_difference': 'minus_K_squarred_difference',
 'K_standard_error': 'minus_K_standard_error',
 'K_max': 'minus_K_max',
 'K_min': 'minus_K_min'}

In [14]:
minus = minus.rename(renme, axis=1)
minus.reset_index()

,id,class,minus_D_sum,minus_D_count,minus_D_mean,minus_D_squarred_difference,minus_D_standard_error,minus_D_max,minus_D_min,minus_K_sum,minus_K_count,minus_K_mean,minus_K_squarred_difference,minus_K_standard_error,minus_K_max,minus_K_min
0,0,D,-405.776287,2756.500000,-0.147207,371.038374,0.366952,-2.053801e-06,-0.814834,-367.086575,1339.000000,-0.274150,137.040897,0.320035,-6.324865e-06,-0.879949
1,1,K,-665.306470,2913.833333,-0.228327,117.747959,0.201057,-3.195634e-08,-0.810106,-164.958415,1236.166667,-0.133444,40.737794,0.181608,-2.474555e-07,-0.729862
2,2,D,-349.802807,2779.833333,-0.125836,145.560923,0.228871,-5.146908e-07,-0.750359,-226.382758,1319.666667,-0.171545,58.973844,0.211477,-8.738447e-07,-0.809787
3,3,K,-451.004127,2890.333333,-0.156039,66.458261,0.151662,-1.918231e-06,-0.750717,-135.502683,1256.000000,-0.107884,24.595508,0.139993,-7.686410e-07,-0.609637
4,4,D,-445.659327,2794.000000,-0.159506,389.499520,0.373438,-5.177766e-07,-0.766156,-381.058482,1352.333333,-0.281779,145.773365,0.328441,-8.311081e-07,-0.897633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,NaN,-436.845293,2759.333333,-0.158316,476.443660,0.415607,-3.606559e-08,-0.885376,-412.016401,1344.333333,-0.306484,169.876572,0.355611,-8.848775e-08,-0.880758
14996,14996,NaN,-552.881760,2931.500000,-0.188600,74.764719,0.159727,-3.317070e-07,-0.774567,-137.054831,1231.166667,-0.111321,27.121722,0.148483,-4.580895e-06,-0.587979
14997,14997,NaN,-487.924329,2918.500000,-0.167183,79.363371,0.164932,-5.555267e-06,-0.673481,-171.393176,1261.166667,-0.135900,36.499731,0.170189,-1.715604e-07,-0.674133
14998,14998,NaN,-455.885051,2907.833333,-0.156778,125.703597,0.207952,-2.110227e-07,-0.704602,-214.564370,1316.666667,-0.162960,53.003210,0.200714,-5.122261e-06,-0.787857


In [15]:
plus.reset_index()

,id,class,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,K_squarred_difference,K_standard_error,K_max,K_min
0,0,D,777.614282,2975.333333,0.261354,146.102883,0.221633,0.934560,8.721873e-06,189.940639,1218.666667,0.155859,51.886187,0.206425,0.708587,5.579978e-06
1,1,K,379.111613,2749.833333,0.137867,309.572835,0.335589,0.919535,8.830171e-08,320.586678,1293.166667,0.247908,118.153121,0.302387,0.712269,3.714698e-07
2,2,D,464.316126,2942.166667,0.157814,80.203118,0.165134,0.888571,6.482814e-07,156.871831,1235.500000,0.126970,38.416845,0.176407,0.678869,4.606541e-06
3,3,K,294.214732,2791.833333,0.105384,126.880890,0.213222,0.663689,6.930774e-07,211.299259,1292.666667,0.163460,56.289564,0.208756,0.838197,9.555784e-09
4,4,D,820.776927,3009.000000,0.272774,156.927662,0.228408,0.922264,2.884368e-06,196.406639,1249.500000,0.157188,54.868411,0.209637,0.840227,1.146566e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,NaN,892.810953,3019.000000,0.295731,162.570080,0.232092,0.855965,2.215619e-07,194.093758,1227.333333,0.158143,59.578361,0.220414,0.793329,2.313791e-08
14996,14996,NaN,307.139350,2754.333333,0.111511,212.117017,0.277561,0.587058,4.618590e-07,273.660614,1323.500000,0.206770,79.489755,0.245165,0.750197,4.231970e-06
14997,14997,NaN,367.227889,2820.833333,0.130184,125.487351,0.210954,0.709202,5.492385e-06,219.240044,1303.333333,0.168215,52.525153,0.200827,0.612782,2.085316e-06
14998,14998,NaN,471.464040,2966.333333,0.158938,124.351479,0.204781,0.765837,5.575236e-07,215.806324,1298.000000,0.166261,57.210701,0.210024,0.751892,3.505243e-07


In [16]:
Full = pd.merge(plus.reset_index(), minus.reset_index(), how='inner', on=['id', 'class']).set_index('id')
Full

,class,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,...,minus_D_standard_error,minus_D_max,minus_D_min,minus_K_sum,minus_K_count,minus_K_mean,minus_K_squarred_difference,minus_K_standard_error,minus_K_max,minus_K_min
id,,,,,,,,,,,,,,,,,,,,,
0,D,777.614282,2975.333333,0.261354,146.102883,0.221633,0.934560,8.721873e-06,189.940639,1218.666667,...,0.366952,-2.053801e-06,-0.814834,-367.086575,1339.000000,-0.274150,137.040897,0.320035,-6.324865e-06,-0.879949
1,K,379.111613,2749.833333,0.137867,309.572835,0.335589,0.919535,8.830171e-08,320.586678,1293.166667,...,0.201057,-3.195634e-08,-0.810106,-164.958415,1236.166667,-0.133444,40.737794,0.181608,-2.474555e-07,-0.729862
2,D,464.316126,2942.166667,0.157814,80.203118,0.165134,0.888571,6.482814e-07,156.871831,1235.500000,...,0.228871,-5.146908e-07,-0.750359,-226.382758,1319.666667,-0.171545,58.973844,0.211477,-8.738447e-07,-0.809787
3,K,294.214732,2791.833333,0.105384,126.880890,0.213222,0.663689,6.930774e-07,211.299259,1292.666667,...,0.151662,-1.918231e-06,-0.750717,-135.502683,1256.000000,-0.107884,24.595508,0.139993,-7.686410e-07,-0.609637
4,D,820.776927,3009.000000,0.272774,156.927662,0.228408,0.922264,2.884368e-06,196.406639,1249.500000,...,0.373438,-5.177766e-07,-0.766156,-381.058482,1352.333333,-0.281779,145.773365,0.328441,-8.311081e-07,-0.897633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,NaN,892.810953,3019.000000,0.295731,162.570080,0.232092,0.855965,2.215619e-07,194.093758,1227.333333,...,0.415607,-3.606559e-08,-0.885376,-412.016401,1344.333333,-0.306484,169.876572,0.355611,-8.848775e-08,-0.880758
14996,NaN,307.139350,2754.333333,0.111511,212.117017,0.277561,0.587058,4.618590e-07,273.660614,1323.500000,...,0.159727,-3.317070e-07,-0.774567,-137.054831,1231.166667,-0.111321,27.121722,0.148483,-4.580895e-06,-0.587979
14997,NaN,367.227889,2820.833333,0.130184,125.487351,0.210954,0.709202,5.492385e-06,219.240044,1303.333333,...,0.164932,-5.555267e-06,-0.673481,-171.393176,1261.166667,-0.135900,36.499731,0.170189,-1.715604e-07,-0.674133


In [17]:
X, y = Full.loc[0:9999].drop(columns=['class']), Full.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = Full.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=47)
X_train

/tmp/ipykernel_48680/3470367663.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = Full.loc[0:9999].drop(columns=['class']), Full.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,...,minus_D_standard_error,minus_D_max,minus_D_min,minus_K_sum,minus_K_count,minus_K_mean,minus_K_squarred_difference,minus_K_standard_error,minus_K_max,minus_K_min
id,,,,,,,,,,,,,,,,,,,,,
1402,647.190639,2956.833333,0.218880,116.105810,0.198192,0.858153,6.848585e-09,175.884530,1254.000000,0.140259,...,0.280624,-3.896345e-08,-0.877505,-282.637091,1328.333333,-0.212776,88.930929,0.258843,-7.353124e-08,-0.801427
954,674.122952,2994.333333,0.225133,113.644882,0.194849,0.794753,7.374507e-07,171.121413,1229.166667,0.139217,...,0.294891,-3.140496e-07,-0.685993,-299.793299,1342.166667,-0.223365,91.345480,0.260977,-2.986317e-07,-0.774080
6196,896.093021,3043.666667,0.294412,171.046795,0.237099,0.933477,2.664097e-06,192.294736,1234.500000,0.155767,...,0.407716,-1.152291e-06,-0.886074,-413.445182,1368.333333,-0.302152,167.448630,0.349948,-2.143089e-06,-0.869153
6331,296.203553,2706.166667,0.109455,140.330866,0.227761,0.815628,9.935904e-07,212.818351,1264.166667,0.168347,...,0.151039,-2.006735e-07,-0.757134,-128.616960,1214.500000,-0.105901,25.192327,0.144084,-9.433561e-07,-0.655529
7124,836.217539,3015.666667,0.277291,161.741380,0.231628,0.919484,7.279586e-08,187.016247,1227.166667,0.152397,...,0.384141,-3.156265e-06,-0.821102,-381.965775,1355.666667,-0.281755,150.901412,0.333757,-2.284461e-06,-0.848462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2896,674.756683,2947.666667,0.228912,112.598650,0.195479,0.790282,5.685642e-07,163.138617,1209.666667,0.134862,...,0.318994,-2.260297e-06,-0.617497,-310.058906,1311.166667,-0.236476,102.635489,0.279889,-7.883307e-08,-0.664262
8883,408.535379,2876.333333,0.142033,77.582247,0.164262,0.608850,1.240043e-07,169.029254,1264.666667,0.133655,...,0.182866,-6.691764e-08,-0.724837,-184.617924,1287.500000,-0.143393,39.593250,0.175431,-2.128900e-06,-0.707254
6728,374.358749,2825.500000,0.132493,182.241564,0.254011,0.818042,1.802029e-07,253.623731,1313.166667,0.193139,...,0.170478,-8.787665e-07,-0.772408,-158.397385,1269.166667,-0.124804,34.407887,0.164718,-1.029984e-06,-0.717662


In [18]:
X_val

,D_sum,D_count,D_mean,D_squarred_difference,D_standard_error,D_max,D_min,K_sum,K_count,K_mean,...,minus_D_standard_error,minus_D_max,minus_D_min,minus_K_sum,minus_K_count,minus_K_mean,minus_K_squarred_difference,minus_K_standard_error,minus_K_max,minus_K_min
id,,,,,,,,,,,,,,,,,,,,,
7432,711.130310,3002.666667,0.236833,104.845764,0.186894,0.706173,9.371691e-09,168.514607,1226.666667,0.137376,...,0.331761,-5.492700e-07,-0.764417,-337.090598,1349.833333,-0.249728,108.549223,0.283684,-8.947198e-06,-0.685753
5444,822.733848,3040.333333,0.270606,148.701033,0.221191,0.895398,8.391003e-07,190.259241,1248.500000,0.152390,...,0.372602,-9.066728e-07,-0.782061,-379.591284,1359.166667,-0.279282,143.399436,0.324936,-1.108033e-06,-0.915884
5448,637.147331,2968.500000,0.214636,103.376268,0.186644,0.765157,4.088625e-07,173.200377,1233.666667,0.140395,...,0.284299,-1.827204e-06,-0.761788,-285.319944,1334.166667,-0.213856,83.391470,0.250103,-1.254045e-06,-0.759923
8654,379.423348,2738.166667,0.138568,213.063350,0.279000,0.722546,8.426478e-09,264.826107,1276.166667,0.207517,...,0.200728,-1.889903e-07,-0.901316,-175.887113,1234.500000,-0.142476,46.588754,0.194344,-1.791077e-06,-0.853888
7543,696.028596,2962.833333,0.234920,118.651239,0.200150,0.888913,2.669902e-06,176.748397,1229.000000,0.143815,...,0.319921,-1.208898e-06,-0.720933,-323.843411,1339.166667,-0.241825,107.864664,0.283912,-7.448750e-07,-0.837679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6746,977.572108,3012.000000,0.324559,196.589351,0.255520,0.915032,6.255489e-07,211.446100,1220.166667,0.173293,...,0.461502,-8.321547e-07,-0.865584,-456.940194,1346.833333,-0.339270,204.940895,0.390228,-1.013368e-06,-0.914202
2903,559.679999,2948.166667,0.189840,82.943230,0.167760,0.757323,1.077042e-06,156.055630,1258.166667,0.124034,...,0.249886,-3.510263e-07,-0.700484,-257.623676,1322.000000,-0.194874,70.814708,0.231532,-1.410883e-06,-0.790886
3893,647.267784,2973.166667,0.217703,113.632300,0.195530,0.963731,1.241151e-06,172.695380,1226.000000,0.140861,...,0.301912,-1.260832e-06,-0.835951,-304.092706,1338.833333,-0.227133,99.007937,0.272041,-2.071933e-06,-0.808598


In [19]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 5, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61506	validation_1-logloss:0.61709
[1]	validation_0-logloss:0.61221	validation_1-logloss:0.61462
[2]	validation_0-logloss:0.60942	validation_1-logloss:0.61222
[3]	validation_0-logloss:0.60669	validation_1-logloss:0.60983
[4]	validation_0-logloss:0.60404	validation_1-logloss:0.60759
[5]	validation_0-logloss:0.60141	validation_1-logloss:0.60536
[6]	validation_0-logloss:0.59885	validation_1-logloss:0.60318
[7]	validation_0-logloss:0.59630	validation_1-logloss:0.60101
[8]	validation_0-logloss:0.59380	validation_1-logloss:0.59891
[9]	validation_0-logloss:0.59136	validation_1-logloss:0.59683
[10]	validation_0-logloss:0.58894	validation_1-logloss:0.59484
[11]	validation_0-logloss:0.58659	validation_1-logloss:0.59288
[12]	validation_0-logloss:0.58428	validation_1-logloss:0.59097
[13]	validation_0-logloss:0.58200	validation_1-logloss:0.58910
[14]	validation_0-logloss:0.57976	validation_1-logloss:0.58735
[15]	validation_0-logloss:0.57756	validation_1-logloss:0.58554
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [20]:
test_accuracy = accuracy_score(y_val, clf.predict(X_val))
print(f"Test accuracy: {test_accuracy:.12f}")

Test accuracy: 0.780500000000


In [21]:
pred_classes = pd.DataFrame({'id': np.arange(10000, 15000),
                'class': [np.nan] * 5000})
pred_classes

,id,class
0,10000,NaN
1,10001,NaN
2,10002,NaN
3,10003,NaN
4,10004,NaN
...,...,...
4995,14995,NaN
4996,14996,NaN
4997,14997,NaN
4998,14998,NaN


In [22]:
pd.Series(clf.predict(X_test.drop('class', axis=1))).value_counts()

0    4016
1     984
Name: count, dtype: int64

### UP


In [66]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 1000, 10000)
    max_depth = trial.suggest_int('max_depth',  4, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 3)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0.5, 1.0)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0.0, 0.05)
    early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 25, 100)

    clf = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree,
    colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, gamma=gamma, objective='binary:logistic', early_stopping_rounds=early_stopping_rounds, eval_metric = 'logloss')

    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=False,
    )

    score = f1_score(y_true=y_val, y_pred=clf.predict(X_val))

    return score

In [67]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=26), pruner="MedianPruner")

[I 2024-12-01 11:58:44,288] A new study created in memory with name: no-name-fc537e11-3f70-4e57-92a9-afd518526eea


In [68]:
study.optimize(objective, n_trials=100, n_jobs=12)

[I 2024-12-01 11:58:59,757] Trial 3 finished with value: 0.5794573643410853 and parameters: {'n_estimators': 1408, 'max_depth': 4, 'learning_rate': 0.008339194301611246, 'min_child_weight': 1, 'subsample': 0.9422600318031119, 'colsample_bytree': 0.7469201541056706, 'colsample_bylevel': 0.9252528432039986, 'colsample_bynode': 0.6518246224125078, 'gamma': 0.04614397523587103, 'early_stopping_rounds': 43}. Best is trial 3 with value: 0.5794573643410853.
[I 2024-12-01 11:59:01,010] Trial 10 finished with value: 0.5792031098153547 and parameters: {'n_estimators': 1716, 'max_depth': 6, 'learning_rate': 0.009725195330510032, 'min_child_weight': 3, 'subsample': 0.958758157660115, 'colsample_bytree': 0.778538383648939, 'colsample_bylevel': 0.5299050341198681, 'colsample_bynode': 0.7327148918852932, 'gamma': 0.02202310855191867, 'early_stopping_rounds': 93}. Best is trial 3 with value: 0.5794573643410853.
[I 2024-12-01 11:59:02,767] Trial 5 finished with value: 0.5756097560975609 and parameters:

In [32]:
study.best_value

0.788

In [30]:
import pickle
with open("optustudy.pkl", "wb") as f:
    pickle.dump(study, f)

In [24]:
import pickle
with open("optustudy.pkl", "rb") as f:
    study = pickle.load(f)

In [33]:
import pickle
best_params = study.best_params
best_params
with open("best_params.pkl", "wb") as f:
    pickle.dump(best_params, f)

In [29]:
import pickle
with open("best_params.pkl", "rb") as f:
    best_params = pickle.load(f)

In [62]:
study.best_params

{'n_estimators': 1929,
 'max_depth': 4,
 'learning_rate': 0.001125039184759245,
 'min_child_weight': 1,
 'subsample': 0.885949084014642,
 'colsample_bytree': 0.9560071120479613,
 'colsample_bylevel': 0.997072923857895,
 'colsample_bynode': 0.6949763006988456,
 'gamma': 0.034608734040679295,
 'early_stopping_rounds': 35}

In [128]:
best_params = {'n_estimators': 3922,
 'max_depth': 5,
 'learning_rate': 0.009907612050213337,
 'min_child_weight': 2,
 'subsample': 0.8207409625227464,
 'colsample_bytree': 0.9681717244513517,
 'colsample_bylevel': 0.9939001281382152,
 'colsample_bynode': 0.7464015389223911,
 'gamma': 0.00327149407689738,
 'early_stopping_rounds': 98}
best_params

{'n_estimators': 3922,
 'max_depth': 5,
 'learning_rate': 0.009907612050213337,
 'min_child_weight': 2,
 'subsample': 0.8207409625227464,
 'colsample_bytree': 0.9681717244513517,
 'colsample_bylevel': 0.9939001281382152,
 'colsample_bynode': 0.7464015389223911,
 'gamma': 0.00327149407689738,
 'early_stopping_rounds': 98}

In [129]:
clf = XGBClassifier(**best_params, eval_metric = 'logloss', n_jobs=-1)
clf.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61713
[1]	validation_0-logloss:0.61471
[2]	validation_0-logloss:0.61238
[3]	validation_0-logloss:0.61001
[4]	validation_0-logloss:0.60776
[5]	validation_0-logloss:0.60562
[6]	validation_0-logloss:0.60342
[7]	validation_0-logloss:0.60142
[8]	validation_0-logloss:0.59939
[9]	validation_0-logloss:0.59734
[10]	validation_0-logloss:0.59530
[11]	validation_0-logloss:0.59332
[12]	validation_0-logloss:0.59148
[13]	validation_0-logloss:0.58960
[14]	validation_0-logloss:0.58778
[15]	validation_0-logloss:0.58598
[16]	validation_0-logloss:0.58427
[17]	validation_0-logloss:0.58253
[18]	validation_0-logloss:0.58079
[19]	validation_0-logloss:0.57908
[20]	validation_0-logloss:0.57735
[21]	validation_0-logloss:0.57576
[22]	validation_0-logloss:0.57418
[23]	validation_0-logloss:0.57268
[24]	validation_0-logloss:0.57114
[25]	validation_0-logloss:0.56973
[26]	validation_0-logloss:0.56829
[27]	validation_0-logloss:0.56689
[28]	validation_0-logloss:0.56547
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.9939001281382152,
              colsample_bynode=0.7464015389223911,
              colsample_bytree=0.9681717244513517, device=None,
              early_stopping_rounds=98, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=0.00327149407689738, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.009907612050213337,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3922, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [130]:
f1_score(y_val,clf.predict(X_val))

0.586977648202138

In [131]:
accuracy_score(y_val,clf.predict(X_val))

0.7875

In [132]:
pred_classes['class'] = clf.predict(X_test.drop('class', axis=1))
pred_classes['class'].value_counts()

class
0    3961
1    1039
Name: count, dtype: int64

In [133]:
pred_classes.replace({0: 'D', 1: 'K'}).to_csv('full_ml_optu4.csv', index=False)

In [134]:
pred_classes['class'] = clf.predict_proba(X_test.drop('class', axis=1))[:, 1]
pred_classes.loc[:, 'class'] = pred_classes['class'].apply(lambda x: x > 0.45)

/tmp/ipykernel_48680/4065799902.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False  True False ... False False  True]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  pred_classes.loc[:, 'class'] = pred_classes['class'].apply(lambda x: x > 0.45)


In [135]:
pred_classes.loc[:, 'class'].value_counts()

class
False    3812
True     1188
Name: count, dtype: int64

In [137]:
pred_classes.replace({False: 'D', True: 'K'}).to_csv('best_predict.csv', index=False)

# CatBoost


In [32]:
from catboost import CatBoostClassifier

In [ ]:
def objective(trial):
    iterations = trial.suggest_int('iterations', 4500, 25000)
    depth = trial.suggest_int('depth',  4, 8)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)
    min_child_samples = trial.suggest_int('min_child_samples', 1, 3)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0.5, 1.0)
    early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 25, 100)

    clf = CatBoostClassifier(iterations=iterations, depth=depth, learning_rate=learning_rate, min_child_samples=min_child_samples, subsample=subsample,
    colsample_bylevel=colsample_bylevel, objective='CrossEntropy', early_stopping_rounds=early_stopping_rounds, eval_metric = 'CrossEntropy')

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=False,
    )

    score = accuracy_score(y_true=y_val, y_pred=clf.predict(X_val))

    return score

In [38]:
study1 = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=26), pruner="MedianPruner")

[I 2024-12-01 12:16:30,520] A new study created in memory with name: no-name-104c86f7-6e73-49fa-905b-b9a48030b61f


In [39]:
study1.optimize(objective, n_trials=100, n_jobs=12)

[I 2024-12-01 12:16:43,834] Trial 1 finished with value: 0.787 and parameters: {'iterations': 5602, 'depth': 4, 'learning_rate': 0.007303315500705453, 'min_child_samples': 3, 'subsample': 0.9599670989529948, 'colsample_bylevel': 0.8262236645189978, 'early_stopping_rounds': 57}. Best is trial 1 with value: 0.787.
[I 2024-12-01 12:16:44,266] Trial 11 finished with value: 0.7835 and parameters: {'iterations': 11686, 'depth': 6, 'learning_rate': 0.009247290754337788, 'min_child_samples': 3, 'subsample': 0.998516383265694, 'colsample_bylevel': 0.5535029598824189, 'early_stopping_rounds': 98}. Best is trial 1 with value: 0.787.
[I 2024-12-01 12:16:51,671] Trial 8 finished with value: 0.785 and parameters: {'iterations': 13082, 'depth': 6, 'learning_rate': 0.00584301064681636, 'min_child_samples': 1, 'subsample': 0.801095182860576, 'colsample_bylevel': 0.9900670180042314, 'early_stopping_rounds': 40}. Best is trial 1 with value: 0.787.
[I 2024-12-01 12:16:55,484] Trial 3 finished with value: 

In [41]:
study1.best_value

0.789

In [42]:
import pickle
with open("optustudy_cb.pkl", "wb") as f:
    pickle.dump(study1, f)

In [ ]:
import pickle
with open("optustudy_cb.pkl", "rb") as f:
    study1 = pickle.load(f)

In [43]:
import pickle
best_params = study1.best_params
best_params
with open("best_params_cb.pkl", "wb") as f:
    pickle.dump(best_params, f)

In [ ]:
import pickle
with open("best_params_cb.pkl", "rb") as f:
    best_params = pickle.load(f)

In [45]:
best_params

{'iterations': 16996,
 'depth': 4,
 'learning_rate': 0.008327915245718697,
 'min_child_samples': 3,
 'subsample': 0.9996174221744758,
 'colsample_bylevel': 0.7274557743508372,
 'early_stopping_rounds': 36}

In [47]:
clf = CatBoostClassifier(**best_params, objective='CrossEntropy', eval_metric = 'CrossEntropy')

clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=True,
    )

0:	learn: 0.6895680	test: 0.6897171	best: 0.6897171 (0)	total: 3.02ms	remaining: 51.4s
1:	learn: 0.6861375	test: 0.6864598	best: 0.6864598 (1)	total: 6.25ms	remaining: 53.2s
2:	learn: 0.6827197	test: 0.6831775	best: 0.6831775 (2)	total: 9.21ms	remaining: 52.1s
3:	learn: 0.6793694	test: 0.6799783	best: 0.6799783 (3)	total: 12.1ms	remaining: 51.6s
4:	learn: 0.6760405	test: 0.6767979	best: 0.6767979 (4)	total: 15.1ms	remaining: 51.2s
5:	learn: 0.6727758	test: 0.6735697	best: 0.6735697 (5)	total: 17.9ms	remaining: 50.6s
6:	learn: 0.6695195	test: 0.6704379	best: 0.6704379 (6)	total: 20.6ms	remaining: 50.1s
7:	learn: 0.6664650	test: 0.6675455	best: 0.6675455 (7)	total: 23ms	remaining: 48.9s
8:	learn: 0.6633947	test: 0.6645653	best: 0.6645653 (8)	total: 25.8ms	remaining: 48.6s
9:	learn: 0.6604302	test: 0.6617010	best: 0.6617010 (9)	total: 28.4ms	remaining: 48.3s
10:	learn: 0.6574625	test: 0.6588687	best: 0.6588687 (10)	total: 31.2ms	remaining: 48.2s
11:	learn: 0.6545329	test: 0.6560467	best: 

In [48]:
f1_score(y_val,clf.predict(X_val)), accuracy_score(y_val,clf.predict(X_val))

(0.5942307692307692, 0.789)

In [49]:
pred_classes['class'] = clf.predict(X_test.drop('class', axis=1))
pred_classes['class'].value_counts()

class
0    3966
1    1034
Name: count, dtype: int64

In [51]:
pred_classes.replace({0: 'D', 1: 'K'}).to_csv('catboost_1.csv', index=False)

In [55]:
pred_classes['class'] = clf.predict_proba(X_test.drop('class', axis=1))[:, 1]
pred_classes.loc[:, 'class'] = pred_classes['class'].apply(lambda x: x > 0.44)

/tmp/ipykernel_48680/3219156837.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False  True False ... False False  True]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pred_classes.loc[:, 'class'] = pred_classes['class'].apply(lambda x: x > 0.44)


In [56]:
pred_classes['class'].value_counts()

class
False    3794
True     1206
Name: count, dtype: int64

In [57]:
pred_classes.replace({False: 'D', True: 'K'}).to_csv('catBoost_post.csv', index=False)

# Combinations (XG, CB)


In [70]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 1000, 10000)
    iterations = trial.suggest_int('iterations', 4500, 25000)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)
    min_child_samples = trial.suggest_int('min_child_samples', 1, 3)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0.0, 0.01)
    early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 25, 100)

    # CatBoost
    cb = CatBoostClassifier(
        iterations=iterations,
        depth=max_depth,
        learning_rate=learning_rate,
        min_child_samples=min_child_samples,
        subsample=subsample,
        colsample_bylevel=colsample_bylevel,
        objective='CrossEntropy',
        early_stopping_rounds=early_stopping_rounds,
        eval_metric='CrossEntropy',
        verbose=False,
    )
    cb.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

    # XGBoost
    xg = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        colsample_bylevel=colsample_bylevel,
        colsample_bynode=colsample_bynode,
        gamma=gamma,
        objective='binary:logistic',
        eval_metric='logloss',
        early_stopping_rounds=early_stopping_rounds,
    )
    xg.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    # Averaging predictions
    predict_cb = cb.predict_proba(X_val)[:, 1]
    predict_xg = xg.predict_proba(X_val)[:, 1]
    predict = (0.2 * predict_cb + 0.8 * predict_xg)

    # Convert to binary predictions
    predict_binary = (predict > 0.5).astype(int)
    score = accuracy_score(y_true=y_val, y_pred=predict_binary)

    return score


In [71]:
study1 = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=26), pruner="MedianPruner")

[I 2024-12-01 12:44:42,206] A new study created in memory with name: no-name-15e66d70-9858-41ea-9da5-5c061ce45ac4


In [72]:
study1.optimize(objective, n_trials=1000, n_jobs=12)

[I 2024-12-01 12:45:01,524] Trial 3 finished with value: 0.784 and parameters: {'n_estimators': 2547, 'iterations': 14290, 'max_depth': 5, 'learning_rate': 0.009751162570502462, 'min_child_samples': 1, 'subsample': 0.8168911555426841, 'colsample_bylevel': 0.8460178041286739, 'colsample_bytree': 0.6986459633866741, 'colsample_bynode': 0.8256189584145079, 'gamma': 0.007009875391852474, 'early_stopping_rounds': 36}. Best is trial 3 with value: 0.784.
[I 2024-12-01 12:45:15,141] Trial 7 finished with value: 0.7805 and parameters: {'n_estimators': 2384, 'iterations': 6925, 'max_depth': 7, 'learning_rate': 0.009988449562489888, 'min_child_samples': 1, 'subsample': 0.8690539397615187, 'colsample_bylevel': 0.6300810819567124, 'colsample_bytree': 0.9778546291500432, 'colsample_bynode': 0.6592027429530944, 'gamma': 0.0037687826702452734, 'early_stopping_rounds': 53}. Best is trial 3 with value: 0.784.
[I 2024-12-01 12:45:19,058] Trial 11 finished with value: 0.781 and parameters: {'n_estimators'

In [73]:
study1.best_value

0.7875

In [74]:
import pickle
with open("optustudy_comb.pkl", "wb") as f:
    pickle.dump(study1, f)

In [ ]:
import pickle
with open("optustudy_comb.pkl", "rb") as f:
    study1 = pickle.load(f)

In [75]:
import pickle
best_params = study1.best_params
best_params
with open("best_params_comb.pkl", "wb") as f:
    pickle.dump(best_params, f)

In [ ]:
import pickle
with open("best_params_comb.pkl", "rb") as f:
    best_params = pickle.load(f)

In [77]:
best_params

{'n_estimators': 6047,
 'iterations': 18634,
 'max_depth': 5,
 'learning_rate': 0.0036706330856838055,
 'min_child_samples': 3,
 'subsample': 0.8335330230996061,
 'colsample_bylevel': 0.7732929741634542,
 'colsample_bytree': 0.9777569752420869,
 'colsample_bynode': 0.5295287028681481,
 'gamma': 0.006701902475954345,
 'early_stopping_rounds': 76}

In [80]:
study1.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bylevel,params_colsample_bynode,params_colsample_bytree,params_early_stopping_rounds,params_gamma,params_iterations,params_learning_rate,params_max_depth,params_min_child_samples,params_n_estimators,params_subsample,state
0,0,0.7810,2024-12-01 12:44:43.366329,2024-12-01 12:47:19.696091,0 days 00:02:36.329762,0.793811,0.842818,0.577579,95,0.008490,6055,0.003891,8,3,7222,0.991661,COMPLETE
1,1,0.7795,2024-12-01 12:44:43.370452,2024-12-01 12:46:14.344682,0 days 00:01:30.974230,0.864784,0.559152,0.554708,56,0.002870,22251,0.005017,8,1,3205,0.835367,COMPLETE
2,2,0.7845,2024-12-01 12:44:43.373626,2024-12-01 12:46:56.893734,0 days 00:02:13.520108,0.619247,0.875516,0.860751,89,0.005311,21313,0.001544,5,1,6708,0.812468,COMPLETE
3,3,0.7840,2024-12-01 12:44:43.376956,2024-12-01 12:45:01.524049,0 days 00:00:18.147093,0.846018,0.825619,0.698646,36,0.007010,14290,0.009751,5,1,2547,0.816891,COMPLETE
4,4,0.7810,2024-12-01 12:44:43.380757,2024-12-01 12:45:28.963902,0 days 00:00:45.583145,0.929883,0.718379,0.535844,73,0.000480,5676,0.007908,7,2,6169,0.895153,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.7815,2024-12-01 14:18:30.611209,2024-12-01 14:19:12.983588,0 days 00:00:42.372379,0.674357,0.960250,0.531352,89,0.005470,15616,0.005918,6,2,8648,0.875994,COMPLETE
996,996,0.7825,2024-12-01 14:18:31.482776,2024-12-01 14:19:03.098502,0 days 00:00:31.615726,0.898787,0.936244,0.772190,99,0.006845,11746,0.005976,4,1,4139,0.823634,COMPLETE
997,997,0.7825,2024-12-01 14:18:35.674094,2024-12-01 14:19:27.973531,0 days 00:00:52.299437,0.686294,0.986078,0.769997,63,0.005354,8242,0.006441,7,1,4872,0.849497,COMPLETE
998,998,0.7840,2024-12-01 14:18:42.340858,2024-12-01 14:19:07.571150,0 days 00:00:25.230292,0.890433,0.748926,0.722737,29,0.001277,24480,0.006532,5,2,5603,0.995590,COMPLETE


In [82]:
xg = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        colsample_bylevel=best_params['colsample_bylevel'],
        colsample_bynode=best_params['colsample_bynode'],
        gamma=best_params['gamma'],
        objective='binary:logistic',
        eval_metric='logloss',
        early_stopping_rounds=best_params['early_stopping_rounds'],
    )
cb = CatBoostClassifier(
        iterations=best_params['iterations'],
        depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        min_child_samples=best_params['min_child_samples'],
        subsample=best_params['subsample'],
        colsample_bylevel=best_params['colsample_bylevel'],
        objective='CrossEntropy',
        early_stopping_rounds=best_params['early_stopping_rounds'],
        eval_metric='CrossEntropy',
        verbose=False,
    )

xg.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)
cb.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=True)

[0]	validation_0-logloss:0.61872
[1]	validation_0-logloss:0.61783
[2]	validation_0-logloss:0.61694
[3]	validation_0-logloss:0.61605
[4]	validation_0-logloss:0.61516
[5]	validation_0-logloss:0.61430
[6]	validation_0-logloss:0.61346
[7]	validation_0-logloss:0.61263
[8]	validation_0-logloss:0.61178
[9]	validation_0-logloss:0.61095
[10]	validation_0-logloss:0.61010
[11]	validation_0-logloss:0.60925
[12]	validation_0-logloss:0.60840
[13]	validation_0-logloss:0.60757
[14]	validation_0-logloss:0.60676
[15]	validation_0-logloss:0.60597
[16]	validation_0-logloss:0.60520
[17]	validation_0-logloss:0.60441
[18]	validation_0-logloss:0.60360
[19]	validation_0-logloss:0.60285
[20]	validation_0-logloss:0.60207
[21]	validation_0-logloss:0.60136
[22]	validation_0-logloss:0.60060
[23]	validation_0-logloss:0.59983
[24]	validation_0-logloss:0.59906
[25]	validation_0-logloss:0.59832
[26]	validation_0-logloss:0.59760
[27]	validation_0-logloss:0.59687
[28]	validation_0-logloss:0.59614
[29]	validation_0-loglos

In [111]:
val_pred_05 = ((0.5 * xg.predict_proba(X_val)[:, 1] + 0.5 * cb.predict_proba(X_val)[:, 1]) > 0.5).astype(int)
val_pred_08 = ((0.8 * xg.predict_proba(X_val)[:, 1] + 0.2 * cb.predict_proba(X_val)[:, 1]) > 0.5).astype(int)
print(f"CatBoost:\nF1: {f1_score(y_val, cb.predict(X_val))}\nAccuracy: {accuracy_score(y_val, cb.predict(X_val))}\n\nXGBoost:\nF1: {f1_score(y_val, xg.predict(X_val))}\nAccuracy: {accuracy_score(y_val, xg.predict(X_val))}\n\n" +
      f"0.5 XGBoost + 0.5 CatBoost:\nF1: {f1_score(y_val, val_pred_05)}\nAccuracy: {accuracy_score(y_val, val_pred_05)}\n\nCatBoost * 0.2 + XGBoost * 0.8:\nF1: {f1_score(y_val, val_pred_08)}\nAccuracy: {accuracy_score(y_val, val_pred_08)}")

CatBoost:
F1: 0.5876685934489403
Accuracy: 0.786

XGBoost:
F1: 0.586977648202138
Accuracy: 0.7875

0.5 XGBoost + 0.5 CatBoost:
F1: 0.5847047434656341
Accuracy: 0.7855

CatBoost * 0.2 + XGBoost * 0.8:
F1: 0.5877788554801164
Accuracy: 0.7875


In [120]:
prediction = ((0.8 * xg.predict_proba(X_test.drop('class', axis=1))[:, 1] + 0.2 * cb.predict_proba(X_test.drop('class', axis=1))[:, 1]) > 0.5).astype(int)
pd.Series(prediction).value_counts()

0    3966
1    1034
Name: count, dtype: int64

In [123]:
prediction = ((0.8 * xg.predict_proba(X_test.drop('class', axis=1))[:, 1] + 0.2 * cb.predict_proba(X_test.drop('class', axis=1))[:, 1]) > 0.44).astype(int)
pd.Series(prediction).value_counts()

0    3795
1    1205
Name: count, dtype: int64

In [126]:
pred_classes['class'] = prediction
pred_classes.replace({0: "D", 1: "K"})

,id,class
0,10000,D
1,10001,K
2,10002,D
3,10003,D
4,10004,D
...,...,...
4995,14995,D
4996,14996,K
4997,14997,D
4998,14998,D


In [127]:
pred_classes.replace({0: "D", 1: "K"}).to_csv('latest_pred_combinatedmodels.csv', index=False)